In [45]:
import pandas as pd

In [46]:
df = pd.DataFrame({'address':["Am S Bahnhof Sundgauer Str , Berlin Zehlendorf",
                             "Müggelstraße 9, Berlin Friedrichshain",
                             "Brachvogelstraße 8, Berlin Kreuzberg"]})

In [47]:
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim

locator = Nominatim(user_agent='myGeocoder')
# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df['location'] = df['address'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df['location'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None).tolist()
# 4 - Remove entries without detectable location
df = df.dropna(subset=['location'])
# 5 - split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['location'].tolist(), index=df.index)
# 6 - Remove unnecessary columns
df = df.drop(columns=['location', 'altitude'])

In [49]:
df

,address,latitude,longitude
1,"Müggelstraße 9, Berlin Friedrichshain",52.512117,13.470778
2,"Brachvogelstraße 8, Berlin Kreuzberg",52.495583,13.397825


In [ ]:
from bs4 import BeautifulSoup
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import HardwareType, Popularity
import re
import requests
import time

import pandas as pd
import numpy as np

In [ ]:
def rotate_user_agent():
    """Choose a new random user agent"""
    HEADERS['User-Agent'] = user_agent_rotator.get_random_user_agent()

def remove_prefix(text, prefix):
    """Note that this method can just be replaced by str.removeprefix()
    if the project ever moves to Python 3.9+"""
    if text and text.startswith(prefix):
        return text[len(prefix):]
    return text


entries = []

pages_to_crawl = 1
for page in range(pages_to_crawl):
    # Sleeping for random seconds to avoid overload of requests
#     sleeptime = np.random.uniform(1, 3)
#     print("sleeping for:", sleeptime, "seconds")
#     time.sleep(sleeptime)
    
    
    url = f"https://www.wg-gesucht.de/wg-zimmer-in-Berlin.8.0.1.82.html"
    print(url)
    
    
    # Setup an agent
    user_agent_rotator = UserAgent(popularity=[Popularity.COMMON.value],
                                       hardware_types=[HardwareType.COMPUTER.value])

    # Define the header for request
    HEADERS = {
            'Connection': 'keep-alive',
            'Pragma': 'no-cache',
            'Cache-Control': 'no-cache',
            'Upgrade-Insecure-Requests': '1',
            'User-Agent': user_agent_rotator.get_random_user_agent(),
            'Accept': 'text/html,application/xhtml+xml,application/xml;'
                      'q=0.9,image/webp,image/apng,*/*;q=0.8,'
                      'application/signed-exchange;v=b3;q=0.9',
            'Sec-Fetch-Site': 'none',
            'Sec-Fetch-Mode': 'navigate',
            'Sec-Fetch-User': '?1',
            'Sec-Fetch-Dest': 'document',
            'Accept-Language': 'en-US,en;q=0.9',
        }
    
    # Rotate the agent to avoid getting stuck
    rotate_user_agent()
    
    # Execute request and obtain soup object
    resp = requests.get(url, headers=HEADERS)
    print(f"Got response code {resp.status_code}")
    soup=BeautifulSoup(resp.content, 'html.parser')


    # Find ads
    findings = soup.find_all(lambda e: e.has_attr('id') and e['id'].startswith('liste-'))
    existing_findings = [
      e for e in findings if e.has_attr('class') and not 'display-none' in e['class']
    ]


    base_url = 'https://www.wg-gesucht.de/'
    for row in existing_findings:
        # Ad title
        title_row = row.find('h3', {"class": "truncate_title"})
        title = title_row.text.strip()

        # Ad url
        url = base_url + remove_prefix(title_row.find('a')['href'], "/")

        # Ad image link
        image = re.match(r'background-image: url\((.*)\);',
                         row.find('div', {"class": "card_image"}).find('a')['style'])[1]

        # Room details and address
        detail_string = row.find("div", {"class": "col-xs-11"}).text.strip().split("|")
        details_array = list(map(lambda s: re.sub(' +', ' ',
                                                  re.sub(r'\W', ' ', s.strip())),
                                 detail_string))
        rooms_tmp = re.findall(r'\d Zimmer', details_array[0])
        rooms = rooms_tmp[0][:1] if rooms_tmp else 0
        address = details_array[2] + ', ' + details_array[1]

        # Flatmates
        try:
            flatmates = row.find("div", {"class": "col-xs-11"}).find("span", {"class": "noprint"})['title']
            flatmates_list = [int(n) for n in re.findall('[0-9]+', flatmates)]
        except TypeError:
            flatmates_list = [0,0,0,0]

        # Price
        numbers_row = row.find("div", {"class": "middle"})
        price = numbers_row.find("div", {"class": "col-xs-3"}).text.strip()

        # Size and ad dates
        dates = re.findall(r'\d{2}.\d{2}.\d{4}',
                           numbers_row.find("div", {"class": "text-center"}).text)

        size = re.findall(r'\d{1,4}\sm²',
                          numbers_row.find("div", {"class": "text-right"}).text)
        if len(size) == 0:
            size=['0']
            
        if len(size) == 0:
            size = [0]
        
        ## Find publication date
        # Minutes and hours are in color green (#218700), while days are in color grey (#898989)
        try:
            published_time = row.find("div", {"class": "col-xs-9"})\
        .find("span", {"style": "color: #218700;"}).text
        except:
            published_time = row.find("div", {"class": "col-xs-9"})\
        .find("span", {"style": "color: #898989;"}).text
        
        # Format it for the date ignoring time
        if 'Minuten' in published_time or 'Stunden' in published_time:
            published_time = time.strftime("%d.%m.%Y", time.localtime())
        else:
            published_time = published_time.split(' ')[1]
        
        # Create dataframe with info
        details = {
            'id': int(url.split('.')[-2]),
            'image': image,
            'url': url,
            'title': title,
            'price(euros)': price.split(' ')[0],
            'size(sqm)': re.findall('[0-9]+', size[0]),  #size[0].split(' ')[0],
            'available rooms': rooms,
            'WG size': flatmates_list[0],
            'available spots': flatmates_list[0]-flatmates_list[1]-flatmates_list[2]-flatmates_list[3],
            'male flatmates': flatmates_list[1],
            'female flatmates': flatmates_list[2],
            'diverse flatmates': flatmates_list[3],
            'published on': published_time,
            'address': address,
            'crawler': 'WG-Gesucht'
        }
        if len(dates) == 2:
            details['available from'] = dates[0]
            details['available to'] = dates[1]
        elif len(dates) == 1:
            details['available from'] = dates[0]
            details['available to'] = 'open end'

        entries.append(details)

In [ ]:
df = pd.DataFrame(entries)
df

In [ ]:
row = existing_findings[4]

